# Analysis of the comments on Instagram

# Using ML to conduct sentiment analysis with LinearSVM

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk

In [3]:
comments = pd.read_csv('post_comments_train.csv', delimiter=',')

In [4]:
comments.head()

,id,url,comment_author,contents,time_posted,user_name_post_author,Sentiment
0,318.0,https://www.instagram.com/p/B7ZC-4wFVVH/,ridwnsomawijaya,:fire:,2020-01-16 19:55:34+01,instagram,Positive
1,176.0,https://www.instagram.com/p/B7UBbVHFxH9/,trulytingz,I LOVE YOU,2020-01-14 21:05:57+01,arianagrande,Positive
2,300.0,https://www.instagram.com/p/B7bd1APhTU4/,chien8112,@siaoyisin @jbm_rich 學起來:rolling_on_the_floor_...,2020-01-17 18:28:38+01,instagram,Neutral
3,1280.0,https://www.instagram.com/p/B6Gqwv3AkmM/,official.0341,:face_with_open_mouth::face_with_open_mouth:,2019-12-15T19:06:12.000Z,instagram,Neutral
4,11149.0,https://www.instagram.com/p/B3nEAzdgJVM/,amirhossein_mie,Me models irani,2019-10-14T19:28:22.000Z,nike,Neutral


In [5]:
todrop = ['id', 'url', 'comment_author', 'time_posted', 'user_name_post_author']
comments = comments.drop(columns=todrop)

In [6]:
comments.Sentiment = comments.Sentiment.map({'Negative':0, 'Neutral':1, 'Positive':2})

In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [8]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])

In [9]:
X_train = comments['contents']
Y_train = comments['Sentiment']

In [17]:
text_clf_svm.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf-svm',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  

In [28]:
# importing the test data:
test_data = pd.read_csv('post_comments_test.csv', delimiter=';')
test_data.Sentiment = test_data.Sentiment.map({'Negative':0, 'Neutral':1, 'Positive':2})

In [30]:
X_test = test_data['contents']
Y_test = test_data['Sentiment']

In [31]:
text_clf_svm.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf-svm',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  

In [32]:
predicted_svm = text_clf_svm.predict(X_test)

In [33]:
predicted_svm

array([1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1,
       2, 2, 1, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2])

In [34]:
np.mean(predicted_svm == Y_test)

0.7701149425287356

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3),}

In [37]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [38]:
gs_clf_svm = gs_clf_svm.fit(X_train, Y_train)

In [39]:
gs_clf_svm.best_score_

0.8136414565826332

In [40]:
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [58]:
text_clf_svm = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
                         ('clf-svm', SGDClassifier(alpha=0.001, random_state=42))])

In [59]:
text_clf_svm.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf-svm',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  

In [60]:
predicted_svm = text_clf_svm.predict(X_test)

In [61]:
np.mean(predicted_svm == Y_test)

0.8045977011494253